* 진형님 LDA 참고하여 진행
* 동현님 gensim 파일 참고하여 진행

# 1. 데이터 로드

저장된 데이터 가져오기

In [1]:
import pandas as pd
import re
import os

In [2]:
import matplotlib.pyplot as plt

# # 한글 깨짐 방지
# plt.rcParams['font.family'] = "Malgun Gothic"

# # 마이너스 깨짐 방지
# plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 파일 불러오기 
current_dir = os.path.dirname(os.path.abspath('df_labor_real_cleaned3.csv'))
data_dir = os.path.join(current_dir, '..', 'data')
file_path = os.path.join(data_dir, 'df_labor_real_cleaned3.csv')  # 실제 파일명으로 변경

# CSV 파일 읽기
data = pd.read_csv(file_path)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   판례정보일련번호      5016 non-null   int64 
 1   사건번호          5016 non-null   object
 2   선고            5016 non-null   object
 3   판결유형          5016 non-null   object
 4   판시사항          3725 non-null   object
 5   판결요지          3054 non-null   object
 6   참조조문          3724 non-null   object
 7   참조판례          2450 non-null   object
 8   판례내용          5016 non-null   object
 9   판례내용_상단       5016 non-null   object
 10  판례내용_이유       5012 non-null   object
 11  판례내용_이유(전처리)  5016 non-null   object
 12  판례내용_제거(불용어)  5016 non-null   object
 13  판례내용_이유(불용어)  5012 non-null   object
dtypes: int64(1), object(13)
memory usage: 548.8+ KB


In [5]:
df_labor = data['판례내용_이유(불용어)'].copy()
df_labor.info()

<class 'pandas.core.series.Series'>
RangeIndex: 5016 entries, 0 to 5015
Series name: 판례내용_이유(불용어)
Non-Null Count  Dtype 
--------------  ----- 
5012 non-null   object
dtypes: object(1)
memory usage: 39.3+ KB


# 2. 데이터 전처리

## 2-1 토큰화

* 형태소 분석 및 전처리: Okt를 사용하여 한국어 문서에서 명사만을 추출하고, 불용어를 제거합니다. 또한, 한 글자 단어들은 제거하여 의미가 있는 단어들만 남깁니다.
* 단어 사전 및 코퍼스 생성: Dictionary 객체를 사용해 단어 사전을 만들고, 이를 통해 문서 코퍼스를 bag-of-words 형식으로 변환합니다.
* LDA 모델 학습: 주제 수(num_topics)를 설정한 후 LDA 모델을 학습합니다.
주제별 핵심 단어 출력: print_topics 메서드를 사용하여 각 주제의 상위 단어를 출력합니다.

In [13]:
# 토픽수 5개
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 전처리 수행
df_labor.fillna('',inplace =True)  # NaN 값 제거

# CountVectorizer로 단어 빈도 행렬 생성
vectorizer = CountVectorizer(max_df=0.95, min_df=2)  # 불용어 제거 설정
X = vectorizer.fit_transform(df_labor)

# LDA 모델 생성 및 학습
n_topics = 6  # 원하는 토픽 개수
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

# 주제별 상위 단어 출력
terms = vectorizer.get_feature_names_out()
for idx, topic in enumerate(lda.components_):
    print(f"\n주제 {idx + 1}:")
    print([terms[i] for i in topic.argsort()[:-10 - 1:-1]])  # 상위 10개 단어 출력


주제 1:
['지급', '임금', '근로', '수당', '퇴직금', '소득', '금액', '회사', '퇴직', '근로자']

주제 2:
['징계', '회사', '노동조합', '해고', '노조', '근로자', '단체', '노동', '쟁의', '조합원']

주제 3:
['공무원', '임용', '교원', '국가', '학교', '활동', '교육', '사회', '정보', '거부']

주제 4:
['근로자', '사업', '근로', '계약', '고용', '노동조합', '임금', '근무', '업체', '파견']

주제 5:
['회사', '주식회사', '사업', '주식', '거래', '순번', '명의', '법인', '계좌', '자금']

주제 6:
['보험', '재해', '급여', '망인', '사고', '지급', '장해', '손해', '산업', '병원']
